In [1]:
using Laplacians

In [2]:
include("/Users/spielman/Laplacians/src/flowUtils.jl")

cutCapacity

In [3]:
include("/Users/spielman/Laplacians/src/min_cost_flow.jl")
#include("/Users/spielman/Laplacians/src/primalDualIPM.jl")
#include("/Users/spielman/Laplacians/src/max_flow_IPM.jl")

makeAdj (generic function with 1 method)

We introduce a data type for minimum cost flow problems.
It has an edge list, a vector of capacities of edges, a vector of costs, and a vector of demands.
We test this with the graph on 4 vertices with source 1 and destination 4.
We try to flow 1 unit of flow.  
The capacities on all edges are 0.7.
The costs on route 1-2-4 are 1, and on route 1-3-4 are 2.
So, it should flow most on route 1-2-4, as it does.

In [4]:
edges = [1 2; 2 4; 1 3; 3 4]
caps = ones(4)*0.7
costs = [1.0; 1; 2 ; 2]
dems = [1.0;0;0;-1]

mcfp = MCFproblem(edges, caps, costs, dems)

MCFproblem{Float64,Int64}([1 2; 2 4; 1 3; 3 4],[0.7,0.7,0.7,0.7],[1.0,1.0,2.0,2.0],[1.0,0.0,0.0,-1.0])

In [5]:
sol = min_cost_flow(mcfp)
flow = sol[1]

Iteration 1, ||r_p||=0.094281, ||r_d||=4.626043, mu=2.380000
Iteration 2, ||r_p||=0.005625, ||r_d||=0.276011, mu=0.160090
Iteration 3, ||r_p||=0.000521, ||r_d||=0.025576, mu=0.035827
Iteration 4, ||r_p||=0.000005, ||r_d||=0.000256, mu=0.000358
Iteration 5, ||r_p||=0.000000, ||r_d||=0.000003, mu=0.000004
Iteration 6, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000000
Termination tolerance reached.


4-element Array{Float64,1}:
 0.7
 0.7
 0.3
 0.3

The following cells compute the costs and check that the demands are satisfied.

In [6]:
println("Cost: ", (mcfp.costs'*flow)[1])
println("Min flow: ", minimum(flow))
println("Min slack: ", minimum(mcfp.capacities-flow))

edge_list = mcfp.edge_list
m = size(edge_list,1)
n = maximum(edge_list)
B = sparse(collect(1:m), edge_list[:,1], 1.0, m, n) -
  sparse(collect(1:m), edge_list[:,2], 1.0, m, n)

println("Error on demands: ", sum(abs(B'*flow- dems)))

Cost: 2.599999998893948
Min flow: 0.29999999981555786
Min slack: 1.8414170188663093e-10
Error on demands: 7.371677157586021e-10


Now, let's try computing this using JuMP, an optimization package for Julia.

In [7]:
using JuMP
using Clp

In [8]:
function MCFjump(mcfp::MCFproblem)
    edge_list = mcfp.edge_list
    m = size(edge_list,1)
    n = maximum(edge_list)
    B = sparse(collect(1:m), edge_list[:,1], 1.0, m, n) -
      sparse(collect(1:m), edge_list[:,2], 1.0, m, n)
 

    mod = Model(solver=ClpSolver())
    @variable(mod, x[1:m] >= 0)
    @constraint(mod, x .<= mcfp.capacities)
    @constraint(mod, B'*x .== mcfp.demands)
    @objective(mod, Min, (mcfp.costs'*x)[1])
    @time status = solve(mod)

    f = getvalue(x)
    return f
    
end

MCFjump (generic function with 1 method)

In [9]:
f = MCFjump(mcfp)

  1.275508 seconds (143.88 k allocations: 6.306 MB)


4-element Array{Float64,1}:
 0.7
 0.7
 0.3
 0.3

We would like to compare the performance of our code to standard codes on some benchmark examples.  These benchmark examples usually come in the Dimacs format.  
In this notebook, we use the example "goto_8_08a.min"
from <a href="http://lime.cs.elte.hu/~kpeter/data/mcf/goto/">http://lime.cs.elte.hu/~kpeter/data/mcf/goto/</a>.

In [10]:
mcfp = readDimacsMCF("/Users/spielman/tmp/goto_8_08a.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 180 216; 216 252],[8642.0,741.0,453.0,62.0,341.0,1029.0,2640.0,1762.0,155.0,31.0  …  55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0],[28.0,714.0,451.0,822.0,0.0,7.0,518.0,29.0,298.0,973.0  …  142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0],[55614.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,-55614.0,0.0,0.0,0.0,0.0])

To run our code on this problem, we need to set a higher tolerance than we usually do.
You will see in a moment that the code generates errors if we do not do this.

In [11]:
sol = min_cost_flow(mcfp, tol=1e-2)
flow = sol[1]

println("Cost: ", (mcfp.costs'*flow)[1])
println("Min flow: ", minimum(flow))
println("Min slack: ", minimum(mcfp.capacities-flow))

edge_list = mcfp.edge_list
m = size(edge_list,1)
n = maximum(edge_list)
B = sparse(collect(1:m), edge_list[:,1], 1.0, m, n) -
  sparse(collect(1:m), edge_list[:,2], 1.0, m, n)

println("Error on demands: ", sum(abs(B'*flow- mcfp.demands)))

Iteration 1, ||r_p||=82578.653557, ||r_d||=1092356.429364, mu=629220336.199898
Iteration 2, ||r_p||=82564.840913, ||r_d||=1092173.714707, mu=629298827.556181
Iteration 3, ||r_p||=82535.269542, ||r_d||=1091782.542463, mu=629463964.755363
Iteration 4, ||r_p||=82436.875911, ||r_d||=1090480.984355, mu=629993065.063792
Iteration 5, ||r_p||=82134.463833, ||r_d||=1086480.655419, mu=631435849.981872
Iteration 6, ||r_p||=81100.918447, ||r_d||=1072808.841952, mu=635028176.950339
Iteration 7, ||r_p||=77271.165564, ||r_d||=1022148.592548, mu=640629738.277029
Iteration 8, ||r_p||=61444.298970, ||r_d||=812789.651283, mu=625107449.020031
Iteration 9, ||r_p||=24622.141586, ||r_d||=325703.477930, mu=426319487.000265
Iteration 10, ||r_p||=10954.931848, ||r_d||=144912.634461, mu=288125231.809748
Iteration 11, ||r_p||=912.478442, ||r_d||=12070.331131, mu=28295686.973661
Iteration 12, ||r_p||=72.245337, ||r_d||=955.666571, mu=2616627.991457
Iteration 13, ||r_p||=34.953746, ||r_d||=462.370698, mu=2395909.66

In [12]:
sol = min_cost_flow(mcfp, tol=1e-6)

Iteration 1, ||r_p||=82578.653557, ||r_d||=1092356.429364, mu=629220336.199898
Iteration 2, ||r_p||=82564.840913, ||r_d||=1092173.714707, mu=629298827.556181
Iteration 3, ||r_p||=82535.269542, ||r_d||=1091782.542463, mu=629463964.755363
Iteration 4, ||r_p||=82436.875911, ||r_d||=1090480.984355, mu=629993065.063792
Iteration 5, ||r_p||=82134.463833, ||r_d||=1086480.655419, mu=631435849.981872
Iteration 6, ||r_p||=81100.918447, ||r_d||=1072808.841952, mu=635028176.950339
Iteration 7, ||r_p||=77271.165564, ||r_d||=1022148.592548, mu=640629738.277029
Iteration 8, ||r_p||=61444.298970, ||r_d||=812789.651283, mu=625107449.020031
Iteration 9, ||r_p||=24622.141586, ||r_d||=325703.477930, mu=426319487.000265
Iteration 10, ||r_p||=10954.931848, ||r_d||=144912.634461, mu=288125231.809748
Iteration 11, ||r_p||=912.478442, ||r_d||=12070.331131, mu=28295686.973661
Iteration 12, ||r_p||=72.245337, ||r_d||=955.666571, mu=2616627.991457
Iteration 13, ||r_p||=34.953746, ||r_d||=462.370698, mu=2395909.66

LoadError: Base.LinAlg.PosDefException(254)

As a sanity check, we compare the results from MCFjump.

In [13]:
f = MCFjump(mcfp)
mcfp.costs'*f

  0.016440 seconds (104 allocations: 674.000 KB)


1-element Array{Float64,1}:
 5.60871e8

This is very similar.  But, JuMP computed a flow of a slightly lower cost.
This should not be surprising, because we haven't figured out exactly what tol we want.
It might help to know that all the numbers in the Dimacs problems are integers.

In [14]:
mcfp.costs'*f - mcfp.costs'*flow

1-element Array{Float64,1}:
 -3.80888

For comparison, we should also try the code from "Lemon", available at

<a href="http://lemon.cs.elte.hu/trac/lemon">http://lemon.cs.elte.hu/trac/lemon</a>

It was easy to install on my Mac (I just needed cmake, which I got by `brew install cmake`)

Here is a transcript of running it:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_08a.min 
Problem type: min
Num of nodes: 256
Num of arcs:  2048

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00467205s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 7.70092e-05s
Run NetworkSimplex: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00143886s

Feasible flow: found
Min flow cost: 560870539
```